# Robot Grid Navigation Problem

Suppose there is a robot at coordinate $A(0, 0)$ that needs to move to coordinate $B(6, 4)$.  
The robot is programmed to **avoid red-colored coordinates** (obstacles).

<img src="grid.png" alt="Grid with obstacles" width="50%">

*Figure: $6 \times 4$ grid with obstacle nodes (red)*

---

The robot is equipped with two modules for movement:

### Module (a):
- The robot may move **one step vertically or horizontally** per move.
- Examples:
  - $(0, 0) → (1, 0)$
  - $(0, 0) → (0, 1)$

### Module (b):
- The robot may move **one step vertically, horizontally, or diagonally upward**.
- Examples:
  - $(0, 0) → (1, 0)$
  - $(0, 0) → (0, 1)$
  - $(0, 0) → (1, 1)$

---
---

## Problem Statements

### 1. Create a SMT encoding to define the movements for module (a)
 - (i) Number of all possible paths from **$A$ to $B$** (without red nodes).
 - (ii) Show that no valid path includes coordinate **$C$**.

---

### 2. Create a SMT encoding to define the movements for module (b)
 - (i) Total paths from **$A$ to $B$**.
 - (ii) Number of minimal-length paths.
 - (iii) Prove all minimal paths must pass **$C$, $D$, and $E$**.

---

## Notes and Hints

- 🧠 Your answer **must be based solely on SMT**; you are **not allowed** to “directly” apply your mathematical knowledge to find the answer.
- 🐍 You are free to create or use an existing **function or library in Python**.
- 🔢 Define integer variables $x_1, y_1, x_2, y_2, \ldots, x_m, y_m $ to represent the robot’s coordinates.
- ➕ Define the relation or constraint between $x_i$ and $x_{i+1}$, and between $y_i$ and  $y_{i+1}$ according to the movement rule for each module
- 🚩 Define explicit **constraints** for initial and final coordinates


## Solution 
Import the necessary libraries and set up the SMT solver. Then, define the variables according to the problem requirements. We known from module (a) and module (b) that the robot maximum steps is 10 steps which is the sum of the x-axis and y-axis steps. Which is in total, we get 11 coordinates for the robot's path, which are: 

$$(x_0, y_0), (x_1, y_1), \ldots, (x_{10}, y_{10})$$


In [1]:
from z3 import *

max_steps = 10

X = [Int(f'x_{i}') for i in range(max_steps+1)]
Y = [Int(f'y_{i}') for i in range(max_steps+1)]

print(", ".join([f"({x}, {y})" for x, y in zip(X, Y)]))

s = Solver()

(x_0, y_0), (x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4), (x_5, y_5), (x_6, y_6), (x_7, y_7), (x_8, y_8), (x_9, y_9), (x_10, y_10)


Set the initial and final coordinates of the robot, which are $A(0, 0)$ and $B(6, 4)$.

$$(x_0, y_0) = (0, 0)$$
$$(x_{10}, y_{10}) = (6, 4)$$

In [2]:
s.add(X[0] == 0, Y[0] == 0)  
s.add(X[max_steps] == 6, Y[max_steps] == 4)  

print(s)

[x_0 == 0, y_0 == 0, x_10 == 6, y_10 == 4]


Since coordinates are non-negative integers, we must ensure that the robot does not move outside the grid. So add the constraints

$$0 \leq x_i \leq 6, \quad 0 \leq y_i \leq 4 \quad\text{where}\quad i\in\{0,1,2,...,10\}$$

In [3]:
s.add([X[i+1] >= 0 for i in range(max_steps)])
s.add([X[i+1] <= 6 for i in range(max_steps)])
s.add([Y[i+1] >= 0 for i in range(max_steps)])
s.add([Y[i+1] <= 4 for i in range(max_steps)])

print(" , ".join([str(c) for c in s.assertions()]))

x_0 == 0 , y_0 == 0 , x_10 == 6 , y_10 == 4 , x_1 >= 0 , x_2 >= 0 , x_3 >= 0 , x_4 >= 0 , x_5 >= 0 , x_6 >= 0 , x_7 >= 0 , x_8 >= 0 , x_9 >= 0 , x_10 >= 0 , x_1 <= 6 , x_2 <= 6 , x_3 <= 6 , x_4 <= 6 , x_5 <= 6 , x_6 <= 6 , x_7 <= 6 , x_8 <= 6 , x_9 <= 6 , x_10 <= 6 , y_1 >= 0 , y_2 >= 0 , y_3 >= 0 , y_4 >= 0 , y_5 >= 0 , y_6 >= 0 , y_7 >= 0 , y_8 >= 0 , y_9 >= 0 , y_10 >= 0 , y_1 <= 4 , y_2 <= 4 , y_3 <= 4 , y_4 <= 4 , y_5 <= 4 , y_6 <= 4 , y_7 <= 4 , y_8 <= 4 , y_9 <= 4 , y_10 <= 4


Add the obstacle coordinates (six obstacles) as constraints.

$$\text{Obs}:=\{(3, 1), (4, 1), (4, 2), (2,2), (2,3) , (3,3)\}$$

The boolean expression for the obstacle coordinates is defined as follows:

$$\left\{(x_i \ne m)\lor (y_i\ne n)\,|\,(m,n)\in\text{Obs},\,i\in\{1,2,\dots,10\}\right\}$$



In [4]:
obs = [(3, 1), (4, 1), (4, 2), (2, 2), (2, 3), (3, 3)]
s.add([Or(X[i+1] != m, Y[i+1] != n) for i in range(max_steps-1) for (m, n) in obs])


print(" , ".join([str(c) for c in s.assertions()]))

x_0 == 0 , y_0 == 0 , x_10 == 6 , y_10 == 4 , x_1 >= 0 , x_2 >= 0 , x_3 >= 0 , x_4 >= 0 , x_5 >= 0 , x_6 >= 0 , x_7 >= 0 , x_8 >= 0 , x_9 >= 0 , x_10 >= 0 , x_1 <= 6 , x_2 <= 6 , x_3 <= 6 , x_4 <= 6 , x_5 <= 6 , x_6 <= 6 , x_7 <= 6 , x_8 <= 6 , x_9 <= 6 , x_10 <= 6 , y_1 >= 0 , y_2 >= 0 , y_3 >= 0 , y_4 >= 0 , y_5 >= 0 , y_6 >= 0 , y_7 >= 0 , y_8 >= 0 , y_9 >= 0 , y_10 >= 0 , y_1 <= 4 , y_2 <= 4 , y_3 <= 4 , y_4 <= 4 , y_5 <= 4 , y_6 <= 4 , y_7 <= 4 , y_8 <= 4 , y_9 <= 4 , y_10 <= 4 , Or(x_1 != 3, y_1 != 1) , Or(x_1 != 4, y_1 != 1) , Or(x_1 != 4, y_1 != 2) , Or(x_1 != 2, y_1 != 2) , Or(x_1 != 2, y_1 != 3) , Or(x_1 != 3, y_1 != 3) , Or(x_2 != 3, y_2 != 1) , Or(x_2 != 4, y_2 != 1) , Or(x_2 != 4, y_2 != 2) , Or(x_2 != 2, y_2 != 2) , Or(x_2 != 2, y_2 != 3) , Or(x_2 != 3, y_2 != 3) , Or(x_3 != 3, y_3 != 1) , Or(x_3 != 4, y_3 != 1) , Or(x_3 != 4, y_3 != 2) , Or(x_3 != 2, y_3 != 2) , Or(x_3 != 2, y_3 != 3) , Or(x_3 != 3, y_3 != 3) , Or(x_4 != 3, y_4 != 1) , Or(x_4 != 4, y_4 != 1) , Or(x_4 != 

### Module (a)
Define relations and constraints for the robot's movements based on the modules described. So we can represented by
$$\begin{align*}
\text{Right step:}&&\text{Up step:}&\\
x_{i+1} &= x_i + 1 &\qquad\qquad\qquad\qquad x_{i+1} &= x_i\\
y_{i+1} &= y_i  & y_{j+1} &= y_i + 1
\end{align*}$$

or in boolean notation we can write
$$\begin{align*}
\text{Right step: }& (x_{i+1} = x_i + 1) \land (y_{i+1} = y_i)\\
\text{Up step: }& (x_{i+1} = x_i) \land (y_{i+1} = y_i + 1)
\end{align*}$$

for $i = 1, 2, \ldots, 6$.

Now because in any position the robot can only move in one of the two directions, so we can define a boolean variable $m_i$ for each step

$$m_i = [(x_{i+1} = x_i + 1) \land (y_{i+1} = y_i)] \lor [(x_{i+1} = x_i) \land (y_{i+1} = y_i + 1)]$$


In [5]:
def right_step(i):
    return And(X[i+1] == X[i] + 1, Y[i+1] == Y[i])

def up_step(i):
    return And(X[i+1] == X[i], Y[i+1] == Y[i] + 1)

In [6]:
def module_a(solver):
    new_solver = Solver()
    new_solver.add(solver.assertions())
    for i in range(max_steps):
        new_solver.add(Or(
            right_step(i),
            up_step(i)
        ))
    return new_solver
    

#### Total possible paths (i)

*There I want to define the function to print all possible paths*


In [7]:
def print_all_paths(solver,c):
    count = c
    s_local = Solver()
    s_local.append(solver.assertions())
    while s_local.check() == sat:
        m = s_local.model()
        path = [(m.evaluate(X[i]).as_long(), m.evaluate(Y[i]).as_long()) for i in range(max_steps+1)]
        print(f"path {count+1} :", path)

        # Block current model
        s_local.add(Or([
            Or(X[i] != m.evaluate(X[i]), Y[i] != m.evaluate(Y[i]))
            for i in range(max_steps+1)
        ]))
        count += 1
    return count
    

In [8]:
solver_module_a = module_a(s)
count_module_a = print_all_paths(solver_module_a,c=0)
print(f"Total number of paths: {count_module_a}")

path 1 : [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4)]
path 2 : [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]
path 3 : [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]
path 4 : [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]
path 5 : [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]
path 6 : [(0, 0), (0, 1), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]
path 7 : [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (6, 1), (6, 2), (6, 3), (6, 4)]
path 8 : [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (6, 2), (6, 3), (6, 4)]
path 9 : [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (6, 4)]
path 10 : [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (5, 3), (6, 3), (6, 4)]
Total number of pat

#### Show that no valid path includes coordinate $C$.

We just need to add the constraint that the robot must be pass coordinate $C(2,1)$, so we can write the boolean expression as follows:
$$\left\{(x_i = 2) \land (y_i = 1)\,|\,\forall i\in\{1,2,\dots,10\}\right\}$$

But we can check if the model is unsatisfiable, which means that there is no valid path that includes coordinate $C$.

In [9]:
C = (2, 1)

solver_module_a.add(Or([And(X[i] == C[0], Y[i] == C[1]) for i in range(max_steps + 1)]))

if solver_module_a.check() == unsat:
  print(f"No valid path includes coordinate C{C}.")
else:
  print(f"There is a valid path that includes coordinate C{C}.")

No valid path includes coordinate C(2, 1).


### Module (b)
Module (b) allows the robot to move in three directions: right, up, and diagonal upward. The **upward diagonal** movement can be represented as follows:
$$\begin{align*}
x_{i+1} &= x_i + 1 \\
y_{i+1} &= y_i + 1 
\end{align*}$$
Thus the boolean variable $m_{i}$ for each step can be defined as follows:
$$m_{i} = [(x_{i+1} = x_i + 1) \land (y_{i+1} = y_i)] \lor [(x_{i+1} = x_i) \land (y_{i+1} = y_i + 1)] \lor [(x_{i+1} = x_i + 1) \land (y_{i+1} = y_i + 1)]$$

In [10]:
def upward_diagonal_step(i):
    return And(X[i+1] == X[i] + 1, Y[i+1] == Y[i] + 1)

#### Total paths from $A$ to $B$ (i)

There is we got some problem because the robot can move in three directions, so the robot have minimum and maximum steps to reach the destination. If we analyze the grid, we can see that the minimum steps is $4+2=6$ steps (4 steps in diagonal and 2 steps in horizontal) and obviously the maximum steps is $10$ steps.

But in this case, we assume that don't know the minimum steps. So the program must be able to check from 1 until 10, but not all of them will be satisfiable. Therefore, if the iteration of minimum steps are satisfiable, we can check the model and print all the paths.

In [11]:
min_steps = 1

def module_b(solver):
    counter = 0
    for i in range(min_steps, max_steps+1):
        new_solver = Solver()
        new_solver.add(solver.assertions())
        new_solver.add(X[i]== 6, Y[i] == 4)  
        new_solver.add([And(X[k] == 0, Y[k] == 0) for k in range(i+1, max_steps)])
        for j in range(i):
            new_solver.add(Or(
                right_step(j),
                up_step(j),
                upward_diagonal_step(j)
            ))
        if new_solver.check() == sat:
            print(f"Module B with {i} steps:")
            counter = print_all_paths(new_solver,counter)
        else:
            print(f"Module B with {i} steps: No valid paths found.")
    print(f"Total number of paths: {counter}")

module_b(s)

Module B with 1 steps: No valid paths found.
Module B with 2 steps: No valid paths found.
Module B with 3 steps: No valid paths found.
Module B with 4 steps: No valid paths found.
Module B with 5 steps: No valid paths found.
Module B with 6 steps:
path 1 : [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 4), (0, 0), (0, 0), (0, 0), (6, 4)]
path 2 : [(0, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 4), (0, 0), (0, 0), (0, 0), (6, 4)]
path 3 : [(0, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 3), (6, 4), (0, 0), (0, 0), (0, 0), (6, 4)]
path 4 : [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 3), (6, 4), (0, 0), (0, 0), (0, 0), (6, 4)]
Module B with 7 steps:
path 5 : [(0, 0), (1, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 3), (6, 4), (0, 0), (0, 0), (6, 4)]
path 6 : [(0, 0), (1, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 4), (0, 0), (0, 0), (6, 4)]
path 7 : [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 3), (6, 3), (6, 4), (0, 0), (0, 0), (6, 4)]
path 8 : [(0, 0), (1, 0), (2, 1), (3, 2), (

#### Number of minimal-length paths (ii)
In the previous section its was seen that the minimum steps is $6$ steps, because for number less than $6$ have no valid path (unsatisfiable).


#### Minimal paths must pass $C$, $D$, and $E$ (iii)
We known that minimal paths is $6$ steps, so if we add the negation of the boolean expression for $C(2,1)$, $D(3,2)$, and $E(4,3)$ as follows:

$$
\begin{align*}
\neg\left[
  ((x_i = 2) \land (y_i = 1)) \lor
  ((x_i = 3) \land (y_i = 2)) \lor
  ((x_i = 4) \land (y_i = 3))
\right],\quad i\in\{1,2,\dots,5\}
\end{align*}
$$

In [12]:
def module_b_ii(solver):
    new_solver = Solver()
    new_solver.add(solver.assertions())
    new_solver.add([Not(Or(And(X[i] == 2,Y[i] == 1),And(X[i] == 3,Y[i] == 2),And(X[i] == 4,Y[i] == 3))) for i in range(1,max_steps)])
    new_solver.add(X[6]== 6, Y[6] == 4)  
    new_solver.add([And(X[k] == 0, Y[k] == 0) for k in range(7, max_steps)])
    for j in range(6):
        new_solver.add(Or(
            right_step(j),
            up_step(j),
            upward_diagonal_step(j)
        ))
    if new_solver.check() == sat:
        print("There exists a minimal path of 6 steps that does NOT pass through C(2,1), D(3,2), or E(4,3):")
        print_all_paths(new_solver, 0)
    else:
        print("Not exist paths of 6 steps without pass through C(2,1), D(3,2), or E(4,3).")

module_b_ii(s)

Not exist paths of 6 steps without pass through C(2,1), D(3,2), or E(4,3).
